# LinkedList ADT (singly linked)
This notebook defines a simple `LinkedList` abstract data type and runs tests.
Upload this `.ipynb` to Google Colab and run all cells.

In [ ]:
from dataclasses import dataclass
from typing import Any, Iterable, Iterator, Optional, Tuple

@dataclass
class Node:
    value: Any
    next: Optional["Node"] = None


class LinkedList:
    """A simple singly linked list ADT."""

    def __init__(self, iterable: Optional[Iterable[Any]] = None):
        self._head: Optional[Node] = None
        self._tail: Optional[Node] = None
        self._size: int = 0
        if iterable is not None:
            for item in iterable:
                self.append(item)

    def __len__(self) -> int:
        return self._size

    def __iter__(self) -> Iterator[Any]:
        cur = self._head
        while cur is not None:
            yield cur.value
            cur = cur.next

    def __repr__(self) -> str:
        return f"LinkedList([{', '.join(repr(x) for x in self)}])"

    def is_empty(self) -> bool:
        return self._size == 0

    def to_list(self) -> list:
        return list(self)

    def append(self, value: Any) -> None:
        """Add to the end."""
        new_node = Node(value)
        if self._head is None:
            self._head = self._tail = new_node
        else:
            assert self._tail is not None
            self._tail.next = new_node
            self._tail = new_node
        self._size += 1

    def prepend(self, value: Any) -> None:
        """Add to the front."""
        new_node = Node(value, next=self._head)
        self._head = new_node
        if self._tail is None:  # list was empty
            self._tail = new_node
        self._size += 1

    def _node_at(self, index: int) -> Node:
        if index < 0 or index >= self._size:
            raise IndexError("index out of range")
        cur = self._head
        for _ in range(index):
            assert cur is not None
            cur = cur.next
        assert cur is not None
        return cur

    def get(self, index: int) -> Any:
        return self._node_at(index).value

    def set(self, index: int, value: Any) -> None:
        self._node_at(index).value = value

    def insert(self, index: int, value: Any) -> None:
        """Insert so that value becomes the element at position index.
        Valid indexes: 0..size (where size means append).
        """
        if index < 0 or index > self._size:
            raise IndexError("index out of range")

        if index == 0:
            self.prepend(value)
            return
        if index == self._size:
            self.append(value)
            return

        prev = self._node_at(index - 1)
        new_node = Node(value, next=prev.next)
        prev.next = new_node
        self._size += 1

    def pop_front(self) -> Any:
        if self._head is None:
            raise IndexError("pop from empty list")
        val = self._head.value
        self._head = self._head.next
        self._size -= 1
        if self._size == 0:
            self._tail = None
        return val

    def pop_back(self) -> Any:
        if self._head is None:
            raise IndexError("pop from empty list")
        if self._size == 1:
            assert self._tail is not None
            val = self._tail.value
            self._head = self._tail = None
            self._size = 0
            return val

        # Walk to node just before tail
        prev = self._node_at(self._size - 2)
        assert prev.next is not None
        val = prev.next.value
        prev.next = None
        self._tail = prev
        self._size -= 1
        return val

    def remove(self, value: Any) -> bool:
        """Remove first occurrence. Returns True if removed, else False."""
        prev: Optional[Node] = None
        cur = self._head
        while cur is not None:
            if cur.value == value:
                if prev is None:
                    # removing head
                    self._head = cur.next
                else:
                    prev.next = cur.next

                if cur.next is None:
                    # removed tail
                    self._tail = prev

                self._size -= 1
                if self._size == 0:
                    self._head = self._tail = None
                return True
            prev = cur
            cur = cur.next
        return False

    def find(self, value: Any) -> int:
        """Return index of first occurrence, or -1 if not found."""
        i = 0
        for x in self:
            if x == value:
                return i
            i += 1
        return -1


## Tests (should all pass)

In [ ]:
def run_tests():
    ll = LinkedList()
    assert ll.is_empty()
    assert len(ll) == 0
    assert ll.to_list() == []

    ll.append(10)
    ll.append(20)
    ll.prepend(5)
    assert ll.to_list() == [5, 10, 20]
    assert len(ll) == 3
    assert ll.get(0) == 5
    assert ll.get(2) == 20

    ll.set(1, 99)
    assert ll.to_list() == [5, 99, 20]

    ll.insert(0, 1)   # front
    ll.insert(2, 7)   # middle
    ll.insert(len(ll), 100)  # end
    assert ll.to_list() == [1, 5, 7, 99, 20, 100]

    assert ll.find(99) == 3
    assert ll.find(12345) == -1

    assert ll.remove(7) is True
    assert ll.to_list() == [1, 5, 99, 20, 100]
    assert ll.remove(7) is False

    assert ll.pop_front() == 1
    assert ll.to_list() == [5, 99, 20, 100]
    assert ll.pop_back() == 100
    assert ll.to_list() == [5, 99, 20]

    # pop to empty
    assert ll.pop_back() == 20
    assert ll.pop_back() == 99
    assert ll.pop_back() == 5
    assert ll.is_empty()

    # errors
    try:
        ll.pop_front()
        assert False, "Expected IndexError"
    except IndexError:
        pass

    try:
        ll.get(0)
        assert False, "Expected IndexError"
    except IndexError:
        pass

    print("✅ All tests passed!")

run_tests()
